In [6]:
# ----------------------------------------------------------------------------
# Dateiname: Teilprüfung_2
# Die Aufgabe besteht aus 2 Teilen:
#   * Theoretischer Teil: 
#     Erklärung, wie die Einführung der bedingten Korrelation und die Verwendung
#     der ReLU-Funktion (Rectified Linear Unit) die Fähigkeit eines tiefen neuronalen Netzes
#     verbessert, komplexe Muster zu erkennen. Beispiel: ein neuronales Netz trainiert wird,
#     um zwischen Bildern von Katzen und Hunden zu unterscheiden.
#   * Praktischer Teil:
#     Python-Skript, das die Initialisierung eines kleinen neuronalen Netzes mit einer 
#     versteckten Schicht zeigt, die ReLU-Funktion anwendet und die bedingte Korrelation illustriert.  
# Autor: Ksenia Meks
# Letzte Änderung: 14.02.2025
# ----------------------------------------------------------------------------

'''  
Bedingte Korrelation durch ReLU - die Logik-Erklärung:

Wenn ein einfaches lineares 2-schichtiges neuronales Netz trainieren wollte, 
anhand der Pixelwerte vorherzusagen, ob auf einem Bild eine Katze zu sehen ist, 
hätte es ein Problem: ein einzelnes Pixel korreliert nicht damit, ob das Bild eine Katze zeigt, 
sondern nur verschiedene Pixelanordnungen.
Das bedeutet, dass es nur einfache Muster wie „Ist das Bild heller oder dunkler?“ 
oder „Ist das Tier eher links oder rechts im Bild?“ erfassen könnte.
Ein neuronales Netz ohne Nichtlinearitäten (also nur mit linearen Aktivierungsfunktionen wie f(x) = x)
könnte nur lineare Zusammenhänge zwischen Eingaben und Ausgaben lernen.

Die Unterscheidung zwischen Katzen und Hunden ist aber ein hochkomplexes Problem, 
das viele nichtlineare Merkmale umfasst, wie Ohrenformen, Felltexturen oder Augenformen. 
Eine lineare Transformation könnte diese Merkmale nicht gut trennen.

Deshalb werden zwischenliegende Schichten (Datenmengen) des Netztes erzeugt, in denen die Knoten 
das Vorhandensein oder das Fehlen einer unterschiedlichen Konfiguration von Eingaben repräsentieren.
Auf diese Weise muss bei der Datenmenge der Katzenbilder ein einzelnes Pixel nicht damit korrelieren, 
ob auf einem Bild eine Katze zu sehen ist, aber stattdessen versucht die mittlere Schicht, verschiedene 
Pixelanordnungen zu identifizieren, die mit dem Vorhandensein einer Katze korrelieren oder nicht.
Beispielweise: das Vorhandensein vieler katzenartiger Pixelanordnungen liefert der letzten Schicht 
die Informationen (die Korrelation), die sie benötigt, um das Vorhandensein oder das Fehlen 
einer Katze richtig vorherzusagen.

In diesem Fall ist es erforderlich, dass diese mittlere Schicht in der Lage ist, selektiv mit der 
Eingabe zu korrelieren, d.h.: 
    die mittlere Schicht soll manchmal mit einer Eingabe korrelieren, manchmal aber auch nicht. 
Das verleiht der mittleren Schicht eine besondere Art der Korrelation: die bedingte Korrelation.

Was dem 3-Schichtigen (zumindest) Netzt ermöglicht, bei Bedarf mit verschiedenen Eingaben zu korrelieren
ist das Deaktivieren eines Knotens der mittleren Schicht, wenn der Wert negativ ist.
Diese Logik, den Wert des Knotens auf 0 zu setzen, wenn er negativ ist, heißt Nichtlinearität. 
Ohne diese Optimierung ist das neuronale Netz linear.

Es gibt viele verschiedene Arten von Nichtlinearitäten und eine von denen ist ReLu.

ReLu (Rectified Linear Unit) ist eine von Aktivierungsfunktionen (d.h. Funktionen, die auf die Neuronen
in einer Schicht während der Vorhersage angewendet werden) und sie bewirkt genau f(x) = max(0, x)
Durch die ReLU-Funktion entstehen regionale Aktivierungen im neuronalen Netz, d. h.: 
    das Netzwerk kann lernen, bestimmte Merkmale nur unter bestimmten Bedingungen zu aktivieren.
Beispiel mit Katzen und Hunden:
*  Eine bestimmte Gruppe von Neuronen könnte lernen, spitze Ohren zu aktivieren 
        → Diese Aktivierung wird durch ReLU auf 0 gesetzt, wenn keine spitzen Ohren erkannt wurden.
*  Eine andere Gruppe könnte lernen, hängende Ohren zu aktivieren 
        → Diese Aktivierung wird durch ReLU auf 0 gesetzt, wenn keine hängende Ohren erkannt wurden.
So entstehen bedingte Korrelationen durch ReLu: 
    Das Netzwerk erkennt nicht nur „irgendwelche Ohren“, sondern spezifisch:
        „spitze Ohren → eher Katze“ oder „hängende Ohren → eher Hund“.
        
Zusammenfassung:
->  Ohne Nichtlinearitäten wie ReLU könnte ein neuronales Netz nur lineare Muster erkennen, 
    was für Katzen/Hunde nicht ausreicht.
->  ReLU macht das Modell nichtlinear und lässt es komplexe Muster erfassen.
->  Die bedingte Korrelation entsteht, weil nur bestimmte Neuronen unter bestimmten Bedingungen 
    aktiviert werden.
->  Das Beispiel in Python zeigt, wie ReLU Werte filtert und dadurch die Fähigkeit des Netzwerks 
    verbessert.
'''

import numpy as np

np.random.seed(42)  # Für reproduzierbare Ergebnisse

# DATEN -> 1000 Messungen mit Zufallsfunktion simuliert

#    Zufällige Generierung von Merkmalen für Katzen und Hunde
spitze_ohren = np.random.uniform(-5, 5, 1000)    # Katzen eher hohe Werte, Hunde niedrig
haengende_ohren = np.random.uniform(-5, 5, 1000) # Hunde hohe Werte, Katzen niedrig
felltextur = np.random.uniform(0, 10, 1000)      # Weiches Fell (Katze) hohe Werte vs. hartes Fell (Hund)
augenform = np.random.uniform(1, 5, 1000)        # Mandelförmig (Katze) niedrige Werte vs. Rund (Hund)

# MATRIX -> Zusammenführen der Daten in eine Matrix 
daten_matrix = np.column_stack((spitze_ohren, haengende_ohren, felltextur, augenform))

# DATEN VORBEREITUNG -> geplante Normalisieren und Aufteilen, Labels

#    Normalisieren der Sensordaten mithilfe der Min-Max-Normalisierung
min_vals = np.min(daten_matrix, axis=0)
max_vals = np.max(daten_matrix, axis=0)
daten_matrix = (daten_matrix - min_vals) / (max_vals - min_vals)

#    Labels generieren (Wahrscheinlichkeit, ob ein Tier eher eine Katze (1) oder ein Hund (0) ist)
#           Spitze Ohren hoch, Hängende Ohren niedrig, Felltextur hoch (weiches Fell), Augenform niedrig -> eher Katze
y_labels =  ((spitze_ohren > 0) & (haengende_ohren < 0) & (felltextur > 5) & (augenform < 3)).astype(int).reshape(-1, 1)

#    Aufteilen der Daten in 80% Trainings- und 20% Testdatensatz
train_size = int(0.8 * len(daten_matrix))
X_train, X_test = daten_matrix[:train_size], daten_matrix[train_size:]
y_train, y_test = y_labels[:train_size], y_labels[train_size:]

# INITIALISIERUN -> Klasse für das neuronale Netz

class NeuronalesNetz:
    def __init__(self, input_size, hidden_size, output_size, alpha=0.05):
        
        self.alpha = alpha
        
        # Gewichte und Biases initialisieren
        self.W1 = np.random.randn(input_size, hidden_size) * 0.2
        self.b1 = np.zeros((1, hidden_size))
        
        self.W2 = np.random.randn(hidden_size, output_size) * 0.2
        self.b2 = np.zeros((1, output_size))

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def forward(self, X):
        # Erste Schicht (Eingabe -> versteckte Schicht)
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)    # ReLu nur für die mittlere Schicht (übliche Vorgehensweise, 
                                        #   besonders im Fall einer binären Klassifikation -> Ausgabe im Bereich [0,1])
                                        # ReLu hilf hier Nichtlinearität zu modellieren
                                        # Es hilt übermäßige Sättigung zu vermeiden und Lernfähigkeit zu erühen
        
        # Zweite Schicht (versteckte Schicht -> Ausgabe)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.z2
        
        return self.a2
    
    def backward(self, X, y):
        m = X.shape[0]  # Anzahl der Trainingsbeispiele
        output = self.forward(X)  # Vorhersage durch das Netz
        error = output - y  # Fehlerberechnung
        
        # Fehler im Output
        dz2 = error
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Fehler in der versteckten Schicht (Backprop durch ReLU)
        dz1 = np.dot(dz2, self.W2.T) * self.relu_derivative(self.z1)  
                                       # ReLu sorgt dafür, dass nur Fehler, 
                                       # die von aktiven Neuronen in der versteckten Schicht kommen, 
                                       # weitergegeben werden -> Lernen-Effizienz
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        # Gewichte aktualisieren
        self.W2 -= self.alpha * dW2
        self.b2 -= self.alpha * db2
        self.W1 -= self.alpha * dW1
        self.b1 -= self.alpha * db1
    
    def train(self, X, y, epochs=1000):
        for i in range(epochs):
            self.backward(X, y)
            if i % 100 == 0:
                loss = np.mean((self.forward(X) - y) ** 2)
                print(f'Epoch {i}, Loss: {loss:.4f}')

    def accuracy(self, X, y):
        predictions = self.forward(X) >= 0.5
        return np.mean(predictions == y) 

# ARCHITEKTUR -> geplante Schichten & Neuronen des tiefen neuronalen Netzes

#    Eingabeschicht:      4 Neuronen: 4 Merkmale hast (Spitze Ohren, Hängende Ohren, Felltextur, Augenform)
input_size = 4 
#    Versteckte Schicht:  8 Neuronen: ein Hyperparameter, den man testen muss. Ich habe die doppelte Anzahl der Eingaben genommen um mehr Kapatzität zu haben
hidden_size = 8
#    Ausgabeschicht:      1 Neuron: Eine binäre Klassifikation (Katze 1, Hund 0) hat normalerweise nur ein einziges Neuron
output_size = 1

# MODELL -> Modell erstellen und trainieren

#    Netz initialisieren
netz = NeuronalesNetz(input_size, hidden_size, output_size)
#    Training
netz.train(X_train, y_train, epochs=1000)

# ERGEBNISSE -> Ausgabe

print(f'Trainingsgenauigkeit: {netz.accuracy(X_train, y_train) * 100:.2f}%')
print(f'Testgenauigkeit: {netz.accuracy(X_test, y_test) * 100:.2f}%')


Epoch 0, Loss: 0.0812
Epoch 100, Loss: 0.0533
Epoch 200, Loss: 0.0522
Epoch 300, Loss: 0.0511
Epoch 400, Loss: 0.0500
Epoch 500, Loss: 0.0490
Epoch 600, Loss: 0.0483
Epoch 700, Loss: 0.0476
Epoch 800, Loss: 0.0471
Epoch 900, Loss: 0.0467
Trainingsgenauigkeit: 93.75%
Testgenauigkeit: 95.00%
